1. Solve the following problems:
    1. $x^3 - 6x^2+11x -6 = 0$

    2. $x = cos(x)$ 

    3. $x_2 = x_1^2-x_1$, $\frac{x_1^2}{16} + x_2^2 =1$ 

    4. $e^{-e^{x_1+x_2}}=x_2(1+x_1^2)$, $x_1\cos(x_2)+x_2\sin(x_1)=\frac{1}{2}$

Problem 1: Roots = [1.0, 2.0, 3.0]  
Problem 2: solution 0.7390851332151607  
Problem 3: Solutions (x1, x2) = [1.58100555, 0.91857299], [-0.61274319,  0.98819741]  
Problem 4: Solution (x1, x2) = [0.44448862, 0.13907865]  

In [18]:
import numpy as np
import scipy as sp

# 1
def f1(x): 
    y1 = x**3 - 6*x**2 + 11*x - 6
    return y1

intervals = [(0.5, 1.5), (1.5, 2.5), (2.5, 3.5)]  # Divide the range into subintervals
roots = []

for x_a, x_b in intervals:
    try:
        root = sp.optimize.bisect(f1, x_a, x_b)
        roots.append(root)
    except ValueError:
        pass  # Skip subintervals where no root is found

print("Problem 1: Roots =", roots)

# 2
def g2(x): 
    y2 = np.cos(x)
    return y2

x_init = 0.5  
x_fixed = sp.optimize.fixed_point(g2, x_init)
print("Problem 2: Fixed point at x =", x_fixed)

# Problem 3: Solve system of equations x2 = x1^2 - x1 and (x1^2)/16 + x2^2 = 1
def g3_system(x):
    x1, x2 = x
    eq1 = x2 - (x1**2 - x1)  # From x2 = x1^2 - x1
    eq2 = (x1**2)/16 + x2**2 - 1  # From (x1^2)/16 + x2^2 = 1
    return [eq1, eq2]

initial_guesses = [[0.5, 0.5], [-0.5, 0.5]]   # Multiple initial guesses
solutions = []

for guess in initial_guesses:
    try:
        sol = sp.optimize.root(g3_system, guess).x
        solutions.append(sol)
    except ValueError:
        pass  # Skip guesses where no solution is found

print("Problem 3: Solutions (x1, x2) =", solutions)


# Problem 4: Solve the system of equations:
# e^(-e^(x1 + x2)) = x2(1 + x1^2)
# x1*cos(x2) + x2*sin(x1) = 1/2
def f4_system(x):
    x1, x2 = x
    eq1 = np.exp(-np.exp(x1 + x2)) - x2 * (1 + x1**2)
    eq2 = x1 * np.cos(x2) + x2 * np.sin(x1) - 0.5
    return [eq1, eq2]

x_init = [0.1, 0.1]  # Initial guess for [x1, x2]
x_solution = sp.optimize.root(f4_system, x_init).x
print("Problem 4: Solution (x1, x2) =", x_solution)

Problem 1: Roots = [1.0, 2.0, 3.0]
Problem 2: Fixed point at x = 0.7390851332151607
Problem 3: Solutions (x1, x2) = [array([1.58100555, 0.91857299]), array([-0.61274319,  0.98819741])]
Problem 4: Solution (x1, x2) = [0.44448862 0.13907865]


2. Question 3.7 in the textbook.

Consider a simple endowment economy with three agents and two goods.  Agent $i$ is initially endowed with $e_{ij}$ units of good $j$ and maximizes utility
$$ U_i(x) = \sum_{j=1}^2 a_{ij} (v_{ij}+1)^{-1} x_{ij}^{v_{ij}+1},$$
subject to the budget constraint
$$ \sum_{j=1}^2 p_j x_{ij} = \sum_{j=1}^2 p_j e_{ij}. $$
Here, $x_{ij}$ is the amount of good $j$ consumed by agent $i$, $p_j$ is the market price of good $j$, and $a_{ij}>0$ and $v_{ij}<0$ are preference parameters.

A competitive general equilibrium for the endowment economy is a pair of relative prices, $p_1$ and $p_2$, normalized to sum to one, such that all the goods markets clear and each agent maximizes utility subject to his budget constraints.  Compute the competitive general equilibrium for the following parameters:

|(i,j)   | a_{ij}   | v_{ij}   | e_{ij} |
|--------|----------|----------|----------|
|(1,1)   | 2.0      | -2.0     |2.0       |
|(1,2)   |1.5       |-0.5      |3.0       |
|(2,1)   |1.5       |-1.5      |1.0       |
|(2,2)   |2.0       |-0.5      |2.0       |
|(3,1)   |1.5       |-0.5      |4.0       |
|(3,2)   |2.0       |-1.5      |0.0       |


Equilibrium Prices: p1 = 0.3393, p2 = 0.6607  
Equilibrium Allocations:  
Agent 1: Good 1 = 3.8617, Good 2 = 2.0441  
Agent 2: Good 1 = 2.0979, Good 2 = 1.4363  
Agent 3: Good 1 = 1.0404, Good 2 = 1.5197  

In [30]:


import numpy as np
from scipy.optimize import root_scalar

# Parameters
a = np.array([[2.0, 1.5], [1.5, 2.0], [1.5, 2.0]])  # a_ij
v = np.array([[-2.0, -0.5], [-1.5, -0.5], [-0.5, -1.5]])  # v_ij
e = np.array([[2.0, 3.0], [1.0, 2.0], [4.0, 0.0]])  # e_ij

# Total endowments of goods
total_endowment = np.sum(e, axis=0)

# Market clearing function
def market_clearing_simple(p1):
    # Normalize prices
    p2 = 1 - p1
    x = np.zeros_like(e)  # Placeholder for equilibrium consumption

    for i in range(3):  # For each agent
        # Compute optimal consumption x_{i1} and x_{i2}
        ratio = (a[i, 1] * p1 / (a[i, 0] * p2)) ** (1 / (v[i, 1] - v[i, 0]))
        budget = p1 * e[i, 0] + p2 * e[i, 1]
        x[i, 0] = budget / (p1 + p2 * ratio)
        x[i, 1] = ratio * x[i, 0]

    # Market clearing conditions for goods 1 and 2
    excess_demand_1 = np.sum(x[:, 0]) - total_endowment[0]
    excess_demand_2 = np.sum(x[:, 1]) - total_endowment[1]
    return [excess_demand_1, excess_demand_2]

# Wrap the function to only return the first market clearing condition (scalar root finding)
def single_equilibrium_condition(p1):
    return market_clearing_simple(p1)[0]

# Solve for equilibrium price p1 using a scalar root-finding approach
result = root_scalar(single_equilibrium_condition, bracket=[0.01, 0.99], method='brentq')

# If successful, compute p1, p2, and equilibrium allocations
if result.converged:
    p1_eq = result.root
    p2_eq = 1 - p1_eq

    # Compute equilibrium allocations
    x_eq = np.zeros_like(e)
    for i in range(3):
        ratio = (a[i, 1] * p1_eq / (a[i, 0] * p2_eq)) ** (1 / (v[i, 1] - v[i, 0]))
        budget = p1_eq * e[i, 0] + p2_eq * e[i, 1]
        x_eq[i, 0] = budget / (p1_eq + p2_eq * ratio)
        x_eq[i, 1] = ratio * x_eq[i, 0]

    # Output results
    print(f"Equilibrium Prices: p1 = {p1_eq:.4f}, p2 = {p2_eq:.4f}")
    print("Equilibrium Allocations:")
    for i in range(3):
        print(f"Agent {i+1}: Good 1 = {x_eq[i, 0]:.4f}, Good 2 = {x_eq[i, 1]:.4f}")
else:
    print("Root-finding failed to converge.")


Equilibrium Prices: p1 = 0.3393, p2 = 0.6607
Equilibrium Allocations:
Agent 1: Good 1 = 3.8617, Good 2 = 2.0441
Agent 2: Good 1 = 2.0979, Good 2 = 1.4363
Agent 3: Good 1 = 1.0404, Good 2 = 1.5197


3. Question 3.8 in the textbook.

Potatoes are storable intraseasonaly, but not interseasonaly, implying that the harvest is entirely consumed over the subsequent twelve months, $i=1,2$. Denoting initial supply by $s$ and consumption in period $i$ by $c_i$, material balance requires that:
$$
s = \sum_{i=1}^{2}c_i.
$$
Competition among storers possessing perfect foresight eliminate arbitrage opportunities; thus,
$$
p_i + \kappa = \delta p_{i+1},\qquad i=1,
$$
where $p_i$ is equilibrium price in period $i$, $\kappa=0.2$ is per-period unit cost of storage, and $\delta=0.95$ is per-period discount factor.  Demand, assumed the same across periods, is given by
$$
p_i=c_i^{-5}.
$$
Compute the equilibrium period 1 and period 2 prices for $s=1$, $s=2$, and $s=2$.


S = 1, Period 1 Price: 31.0925, Period 2 Price: 32.9395  
S = 2, Period 1 Price: 0.8814, Period 2 Price: 1.1383  
S = 3, Period 1 Price: 0.0680, Period 2 Price: 0.2821  

In [34]:
import numpy as np
from scipy.optimize import fsolve

# Parameters
kappa = 0.2  # Per-period unit cost of storage
delta = 0.95  # Per-period discount factor

# Demand function
def demand(c):
    return c**-5

# Equations to solve for equilibrium
def equilibrium_system(variables, s):
    c1, c2 = variables
    p1 = demand(c1)
    p2 = demand(c2)

    eq1 = c1 + c2 - s  # Material balance
    eq2 = p1 + kappa - delta * p2  # Arbitrage condition

    return [eq1, eq2]

# Solve for different values of initial supply
results = {}
for s in [1, 2, 3]:
    initial_guess = [s / 2, s / 2]  # Initial guess for c1 and c2
    solution = fsolve(equilibrium_system, initial_guess, args=(s,))

    c1, c2 = solution
    p1 = demand(c1)
    p2 = demand(c2)

    results[s] = (p1, p2)

# Display results
for s, (p1, p2) in results.items():
    print(f"Supply: {s}, Period 1 Price: {p1:.4f}, Period 2 Price: {p2:.4f}")


Supply: 1, Period 1 Price: 31.0925, Period 2 Price: 32.9395
Supply: 2, Period 1 Price: 0.8814, Period 2 Price: 1.1383
Supply: 3, Period 1 Price: 0.0680, Period 2 Price: 0.2821
